## Testing Transitive Models

This notebook aims to use cross validation to test the performance of transitive models.

### Transitive V1

Transitive V1 is a simple model that instead of predicting matches, aims to predict the probability of a player winning any given point. Then, with dynamic programming we convert the point probabilities to a match win probability. This model only considers two different types of points, service and return points.

We estimate a player's point % win rate against another player by using common opponents. Based on the performance of each player against these common opponents, we estimate the relative serve and return win % between the two plyers.

2024 data is reserved for the validation set.

In [3]:
import numpy as np
import pandas as pd

from atp_forecaster.models.transitive_v1 import TransitiveV1
from atp_forecaster.models.transitive_v2 import TransitiveV2
from atp_forecaster.data.clean import get_cleaned_atp_matches

matches = get_cleaned_atp_matches()
matches[matches['tourney_date'] >= 20240101].head()


,surface,draw_size,tourney_level,tourney_date,id_a,name_a,hand_a,ht_a,age_a,id_b,...,2ndWon_b,SvGms_b,bpSaved_b,bpFaced_b,rank_a,rank_points_a,rank_b,rank_points_b,result,order
88725,Hard,32,A,20240101,209992,Juncheng Shang,L,180.0,18.9,126207,...,11.0,10.0,6.0,8.0,183.0,335.0,16.0,2310.0,1,88725
88726,Hard,32,A,20240101,209950,Arthur Fils,R,185.0,19.5,126094,...,15.0,13.0,7.0,10.0,36.0,1158.0,5.0,4805.0,0,88726
88727,Hard,32,A,20240101,200325,Emil Ruusuvuori,R,188.0,24.7,124116,...,22.0,15.0,9.0,12.0,69.0,771.0,43.0,1048.0,1,88727
88728,Hard,32,A,20240101,209992,Juncheng Shang,L,180.0,18.9,126094,...,19.0,13.0,5.0,8.0,183.0,335.0,5.0,4805.0,0,88728
88729,Hard,32,A,20240101,126094,Andrey Rublev,R,188.0,26.1,200325,...,12.0,10.0,5.0,7.0,5.0,4805.0,69.0,771.0,1,88729


In [4]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score

def test_transitive_v1(model, start_date: int, end_date: int):
    """
    Test the performance of the Transitive V1 model over a given date range.
    """
    # Get all matches between start_date and end_date
    train = matches[matches['tourney_date'] < start_date]
    test = matches[(matches['tourney_date'] >= start_date) & (matches['tourney_date'] < end_date)]

    model.fit(train)

    y_pred = []
    y_true = []

    skipped = 0

    # Predict the outcome of each match in the test set
    for (_, row) in test.iterrows():
        if (len(model.find_common_opponents(row['id_a'], row['id_b'])) < 15):
            skipped += 1
            continue

        prob = model.predict_match(row['id_a'], row['id_b'], best_of=row['best_of'])

        y_pred.append(prob)
        y_true.append(row['result'])

        print(row['id_a'], row['id_b'], row['name_a'], row['name_b'], prob)

        # update model with the match
        model.add_match(row)
    
    y_pred = np.array(y_pred, dtype=float)
    y_true = np.array(y_true, dtype=int)

    loss = log_loss(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    acc = accuracy_score(y_true, (y_pred > 0.5).astype(int))

    coverage = (len(test) - skipped) / len(test)

    return loss, auc, acc, coverage

model_v1 = TransitiveV1(
    look_back=730,
    base_spw=0.6
)

In [11]:
loss, auc, acc, coverage = test_transitive_v1(model_v1, 20230101, 20230131)
print("Log Loss: ", loss)
print("AUC: ", auc)
print("Accuracy: ", acc)
print("Coverage: ", coverage)

200624 104918 Sebastian Korda Andy Murray 0.5902721111567187
106415 208029 Yoshihito Nishioka Holger Rune 0.5369657478985289
106218 104755 Marcos Giron Richard Gasquet 0.5693176984114346
105138 126094 Roberto Bautista Agut Andrey Rublev 0.45117919323194633
200615 200000 Alexei Popyrin Felix Auger Aliassime 0.5464943241931602
106218 200615 Marcos Giron Alexei Popyrin 0.4280901142848157
106415 111456 Yoshihito Nishioka Mackenzie Mcdonald 0.5857205609679792
105138 200624 Roberto Bautista Agut Sebastian Korda 0.47344681326898114
206173 106423 Jannik Sinner Thanasi Kokkinakis 0.31015206152025887
207733 111575 Jack Draper Karen Khachanov 0.515727690001737
106421 200175 Daniil Medvedev Miomir Kecmanovic 0.5550575609928293
106421 132283 Daniil Medvedev Lorenzo Sonego 0.6331168056575032
126127 105936 Benjamin Bonzi Filip Krajinovic 0.46697436441152906
144684 111513 Alex Molcan Laslo Djere 0.5251473015205395
106234 104665 Aslan Karatsev Pablo Andujar 0.5407624462298274
106065 105932 Marco Cecchi

### Transitive V2

Transitive V2 is an improvement over V1, as it considers the surface context of the match. Surface is a huge factor in tennis, and has dramatic impacts on a player's performance. By only considering matches played on the specific surface, we hope to see an increase in predictive power.

In [30]:
def test_transitive_v2(model, start_date: int, end_date: int):
    """
    Test the performance of the Transitive V1 model over a given date range.
    """
    # Get all matches between start_date and end_date
    train = matches[matches['tourney_date'] < start_date]
    test = matches[(matches['tourney_date'] >= start_date) & (matches['tourney_date'] < end_date)]

    model.fit(train)

    y_pred = []
    y_true = []

    skipped = 0

    # Predict the outcome of each match in the test set
    for (_, row) in test.iterrows():
        if (len(model.find_common_opponents(row['id_a'], row['id_b'], surface=row['surface'])) < 15):
            skipped += 1
            continue

        prob = model.predict_match(row['id_a'], row['id_b'], best_of=row['best_of'], surface=row['surface'])

        y_pred.append(prob)
        y_true.append(row['result'])

        print(row['tourney_date'], row['surface'], row['id_a'], row['id_b'], row['name_a'], row['name_b'], prob)

        # update model with the match
        model.add_match(row)
    
    y_pred = np.array(y_pred, dtype=float)
    y_true = np.array(y_true, dtype=int)

    loss = log_loss(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    acc = accuracy_score(y_true, (y_pred > 0.5).astype(int))

    coverage = (len(test) - skipped) / len(test)

    return loss, auc, acc, coverage

model_v2 = TransitiveV2(
    look_back=None,
    base_spw=0.6
)

In [32]:
loss, auc, acc, coverage = test_transitive_v2(model_v2, 20090801, 20091231)
print("Log Loss: ", loss)
print("AUC: ", auc)
print("Accuracy: ", acc)
print("Coverage: ", coverage)



20090802 Hard 103507 102839 Juan Carlos Ferrero Nicolas Lapentti 0.5160518886408462
20090802 Hard 102783 103333 Rainer Schuettler Ivo Karlovic 0.4468984430716293
20090802 Hard 103602 104268 Fernando Gonzalez Alejandro Falla 0.4703980718628548
20090802 Hard 104022 104417 Mikhail Youzhny Robin Soderling 0.5139866622073952
20090802 Hard 103720 104534 Lleyton Hewitt Dudi Sela 0.4551569522909174
20090802 Hard 105223 104229 Juan Martin del Potro Yen Hsun Lu 0.5638289726619181
20090802 Hard 104053 105023 Andy Roddick Sam Querrey 0.5400076395715628
20090802 Hard 103507 103163 Juan Carlos Ferrero Tommy Haas 0.49574953033500074
20090802 Hard 103602 104639 Fernando Gonzalez Wayne Odesnik 0.4068777724442211
20090802 Hard 104417 102967 Robin Soderling Marc Gicquel 0.3963303418613676
20090802 Hard 104053 105223 Andy Roddick Juan Martin del Potro 0.47342554946134635
20090802 Hard 105223 103602 Juan Martin del Potro Fernando Gonzalez 0.5921666918181576
20090802 Hard 104053 103333 Andy Roddick Ivo Karl

In [31]:
model_v2.fit(matches[matches['tourney_date'] < 20090816])

print(len(model_v2.find_common_opponents(104745, 103908, surface='Hard')))
model_v2.predict_match(104745, 103908, surface='Hard')

60


0.3893391886221674